In [37]:
# !pip install pandas
# !pip install xlrd
import pandas as pd

def load_reli(filename):
    try:
        df = pd.read_excel(filename)
        lines = []
        for i in range(0, len(df)):
            if (df.iloc[i, 7] > 0):
                lines.append(df.iloc[i, 1])
        return lines
    except:
        print('Failed to load excel from ' + filename)
        return []

lines = load_reli('/Users/qli/Documents/trusty64/reli/reli_30h剩余数据.csv')
with open('/tmp/test', 'wt') as fout:
    fout.write('\n'.join(lines).encode('utf-8'))

print('Fin!')

Failed to load excel from /Users/qli/Documents/trusty64/reli/reli_30h剩余数据.csv
Fin!


In [44]:
import glob

def load_reli_glob(glob_files):
    lines = []
    for filename in glob.glob(glob_files):
        print('load reli data from ' + filename)
        new_lines = load_reli(filename)
        if lines is not None and len(new_lines) > 0:
            lines = lines + new_lines
        else:
            print(filename + ' contains no reli data')
    return lines

reli_lines = load_reli_glob('/Users/qli/Documents/trusty64/reli/*.csv')
with open('/Users/qli/Documents/trusty64/reli/reli_all.txt', 'wt') as fout:
    fout.write('\n'.join(reli_lines).encode('utf-8'))
print('Fin')

load reli data from /Users/qli/Documents/trusty64/reli/热力23H.csv
load reli data from /Users/qli/Documents/trusty64/reli/reli_30h剩余数据.csv
Failed to load excel from /Users/qli/Documents/trusty64/reli/reli_30h剩余数据.csv
/Users/qli/Documents/trusty64/reli/reli_30h剩余数据.csv contains no reli data
load reli data from /Users/qli/Documents/trusty64/reli/reli_02.csv
load reli data from /Users/qli/Documents/trusty64/reli/reli_03.csv
load reli data from /Users/qli/Documents/trusty64/reli/reli_8k_100h_choose_v2_1.csv
Fin


In [158]:
# install jieba
import jieba

'''
按照字符来分割
'''
def seg_by_char(line):
    line = line.decode('utf8')
    seg_list = []
    for c in line:
        seg_list.append(c)
    return seg_list

def process_reli(filename, segmenter):
    with open(filename, 'rt') as fin:
        lines = [re.sub(r'<.*?>|\[.*?\]', '', line.strip(), flags=re.IGNORECASE) for line in fin]
    segmented_lines = []
    if (segmenter == 'jieba'):
        for line in lines:
            seg_list = jieba.cut(line, cut_all=False)  # 精确模式
            segmented_lines.append(' '.join(seg_list))
    elif (segmenter == 'by_char'):
        for line in lines:
            seg_list = seg_by_char(line)
            segmented_lines.append(' '.join(seg_list))
    else:
        print('unrecognized segmenter ' + segmenter)
        system.exit(1)
    return segmented_lines

corpus_jieba = process_reli('/Users/qli/Documents/trusty64/reli/reli_all.txt', 'jieba')
corpus_by_char = process_reli('/Users/qli/Documents/trusty64/reli/reli_all.txt', 'by_char')

with open('/Users/qli/Documents/trusty64/reli/reli_all.txt.jieba', 'wt') as fout:
    fout.write('\n'.join(corpus_jieba).encode('utf-8'))
    
with open('/Users/qli/Documents/trusty64/reli/reli_all.txt.by_char', 'wt') as fout:
    fout.write('\n'.join(corpus_by_char).encode('utf-8'))

print('Fin')

Fin


In [2]:
# DATA-DRIVEN LEXICON EXPANSION FOR MANDARIN BROADCAST NEWS AND CONVERSATION SPEECH RECOGNITION
import math
import sys
import traceback

class Ngram:
    word_list = []
    count = 0
    
    def __init__(self, word_list):
        self.word_list = word_list
        self.count = 1
        
    def get_n(self):
        return len(self.word_list)
    
    def add_count(self):
        self.count = self.count + 1
        
    def get_count(self):
        return self.count
    
    def get_nth_gram(self, n):
        return self.word_list[n]
    
    def get_word_list(self):
        return self.word_list
    
    def to_string(self):
        return u' '.join(self.word_list)
    
def count_ngram(corpus, n):
    lines = corpus
    ngram_counts = {}
    for line in lines:
        seg_list = line.split(u' ')
        for i in range(0, len(seg_list)):
            if (i + n > len(seg_list)):
                continue
            ngram = Ngram(seg_list[i : i + n])
            lexicon = ngram.to_string()
            # print('lexicon ' + lexicon + ' len ' + str(len(lexicon)))
            if lexicon in ngram_counts:
                ngram_counts[lexicon].add_count()
            else:
                ngram_counts[lexicon] = ngram
    return ngram_counts

def find_ngram(ngram_counts, key):
    if (key in ngram_counts):
        return ngram_counts[key]
    else:
        print('key ' + key + ' doesnt exist in ngram dict')
        traceback.print_stack()
        sys.exit(1)

def compute_LM2_measure(ngram_counts, ngram_wij, ngram_wi, ngram_wj):
    N_wi_wj = ngram_wij.get_count()
    N_wi = ngram_wi.get_count()
    N_wj = ngram_wj.get_count()
    return float(N_wi_wj) / math.sqrt(N_wi * N_wj)
        
def compute_LM3_measure(ngram_counts, ngram):
    if (ngram.get_n() != 3):
        sys.exit(1)
    Ngram_wi_wj = find_ngram(ngram_counts, Ngram(ngram.get_word_list()[0 : 2]).to_string())
    Ngram_wk = find_ngram(ngram_counts, Ngram(ngram.get_word_list()[2 : 3]).to_string())
    Ngram_wj_wk = find_ngram(ngram_counts, Ngram(ngram.get_word_list()[1 : 3]).to_string())
    Ngram_wi = find_ngram(ngram_counts, Ngram(ngram.get_word_list()[0 : 1]).to_string())
    LM2_measure_1 = compute_LM2_measure(ngram_counts, ngram, Ngram_wi_wj, Ngram_wk)
    LM2_measure_2 = compute_LM2_measure(ngram_counts, ngram, Ngram_wi, Ngram_wj_wk)
    return math.sqrt(LM2_measure_1 * LM2_measure_2)

def compute_LM4_measure(ngram_counts, ngram):
    if (ngram.get_n() != 4):
        sys.exit(1)
    Ngram_wi_wj = ngram_counts[Ngram(ngram.get_word_list()[0 : 2]).to_string()]
    Ngram_wk_wl = ngram_counts[Ngram(ngram.get_word_list()[2 : 4]).to_string()]
    return compute_LM2_measure(ngram_counts, ngram, Ngram_wi_wj, Ngram_wk_wl)

def compute_LM(ngram_counts, new_ngram):
    # print('compute_LM 1:' + lexicon + ' len:' + str(len(lexicon_segs)))
    n = new_ngram.get_n()
    if (n == 2):
        Ngram_wi = find_ngram(ngram_counts, Ngram(new_ngram.get_word_list()[0 : 1]).to_string())
        Ngram_wj = find_ngram(ngram_counts, Ngram(new_ngram.get_word_list()[1 : 2]).to_string())
        return compute_LM2_measure(ngram_counts, new_ngram, Ngram_wi, Ngram_wj)
    if (n == 3):
        return compute_LM3_measure(ngram_counts, new_ngram)
    if (n == 4):
        return compute_LM4_measure(ngram_counts, new_ngram)
    else:
        print(u'ERROR: unsupported n ' + str(n))
        sys.exit(0)
        
# corpus: a list of queries
def data_driven_lexicon_discovery(corpus, M, theta, beta):
    N = 4
    new_ngrams = {}  # key = n in ngram, value = new lexicon list
    all_ngram_counts = {}
    for i in range(1, N + 1):
        print('Process ' + str(i) + "-gram")
        ngram_counts = count_ngram(corpus, i)
        
        sorted_ngram_counts = sorted(ngram_counts.items(), key=lambda x: x[1].get_count(), reverse=True)  # sort n-grams by counts

        all_ngram_counts = dict(all_ngram_counts, **ngram_counts)  # dump n-gram counts
        if (i > 1):
            new_ngrams[i] = []
            for j in range(0, M):  # foreach element Wn of the topM n-grams do
                new_ngram = sorted_ngram_counts[j][1]
                score = compute_LM(all_ngram_counts, new_ngram)  # compute the LMn(·) measure
                # print('score = ' + str(score))
                if (score > theta):
                    new_ngrams[i].append(new_ngram)  # put new lexicon at nth list which indicate a lexicon of ngram
    for i in range(N - 1, 1, -1):  # for n ← (N − 1) to 2 do
        for w_n in new_ngrams[i]:  # foreach element Wn of the saved n-grams do
            for w_m in new_ngrams[i + 1]:  # foreach element Wm of the saved higher-order
                if (w_n.to_string() in w_m.to_string()):  # if Wm contains Wn then
                    r = float(all_ngram_counts[w_n].get_count()) / float(all_ngram_counts[w_m].get_count())  # compute counts ratio r of Wn and Wm
                    # print('ratio = ' + str(r))
                    if r < beta:  # if r < β then
                        print('remove ' + w_n)
                        new_ngrams[i].remove(w_n)  # remove Wn from saved n-grams
    for ngrams in new_ngrams.values():
        print('-----------ngram new lexicon-------------')
        print('\n'.join(ngram.to_string() for ngram in ngrams))
            

with open('reli_all.txt.jjieba', 'rt') as fin:
    corpus = [line.decode('utf8').strip() for line in fin]

data_driven_lexicon_discovery(corpus, M = 5000, theta = 0.9, beta = 0)


Process 1-gram
Process 2-gram
Process 3-gram
Process 4-gram
-----------ngram new lexicon-------------
高兴 为您服务
蓝调 沙龙
赛 洛城
珠江 帝景
-----------ngram new lexicon-------------

-----------ngram new lexicon-------------
请问 有 什么 可以
A   B
并 进行 供热 调节
安化 北 里 二号
使用 呼叫 保持 服务
正在 使用 呼叫 保持
